In [1]:

import warnings

import matplotlib.pyplot as plt
# import MultiTrade
import numpy as np
import pandas as pd
from IPython.display import display
from neuralforecast.core import NeuralForecast

from neuralforecast.auto import AutoBiTCN, AutoNBEATS, AutoTFT
from neuralforecast.losses.pytorch import RMSE

# from statsforecast import StatsForecast

from utils.utils import build_market_image,train_test_split_data,plot_insample_forecasts
from configs import defaults
import datetime
warnings.filterwarnings("ignore")

In [2]:
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame
DATA_DIR=defaults.DATA_DIR

In [3]:

horizon = 7
context_length=5*horizon
split_n_days=30
round_time='H'
id_col='symbol'
save_path=defaults.forecasting_model_path

In [4]:

data=build_market_image(target_pair=target_pair,time_frame=time_frame,axis=0,data_dir=DATA_DIR,only_target=True)
# data.index

train_data,test_data=train_test_split_data(data=data,n_days=30)

# data['symbol'].unique()

# test_data
data

['data/binanceus-DOGEUSDT-1h.pkl']


100%|██████████| 78/78 [00:00<00:00, 10987.60it/s]
1it [00:00, 12.19it/s]

2024-10-22 10:00:00 2024-11-21 10:00:00


,open,high,low,close,volume,y,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K,ds,unique_id,symbol
date_close,,,,,,,,,,,,,,,,,,,,
2024-01-01 10:00:00,0.08975,0.08976,0.08975,0.08976,6209.0,0.08976,-0.065217,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2024-01-01 10:00:00,25,DOGEUSDT
2024-01-01 11:00:00,0.08977,0.09012,0.08972,0.09012,13750.0,0.09012,-0.021739,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000008,0.000004,100.000000,0.000000,2024-01-01 11:00:00,25,DOGEUSDT
2024-01-01 12:00:00,0.09008,0.09008,0.08992,0.08992,583.0,0.08992,0.021739,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000004,0.000004,62.566845,0.000000,2024-01-01 12:00:00,25,DOGEUSDT
2024-01-01 13:00:00,0.08995,0.09046,0.08995,0.09040,12750.0,0.09040,0.065217,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000020,0.000009,80.974284,0.000000,2024-01-01 13:00:00,25,DOGEUSDT
2024-01-01 14:00:00,0.09018,0.09053,0.09018,0.09053,5594.0,0.09053,0.108696,-0.5,-0.500000,-0.500000,0.000000,0.000000,0.000000,0.000033,0.000016,83.360761,0.000000,2024-01-01 14:00:00,25,DOGEUSDT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-17 15:00:00,0.36347,0.36425,0.35770,0.36143,223141.0,0.36143,0.152174,0.5,0.033333,0.379452,0.374743,0.361372,0.348002,-0.001727,-0.002703,47.544678,60.921502,2024-11-17 15:00:00,25,DOGEUSDT
2024-11-17 16:00:00,0.36137,0.36550,0.35834,0.36372,137563.0,0.36372,0.195652,0.5,0.033333,0.379452,0.373451,0.360964,0.348477,-0.001471,-0.002457,49.746258,67.434585,2024-11-17 16:00:00,25,DOGEUSDT
2024-11-17 17:00:00,0.36445,0.36762,0.36062,0.36762,196547.0,0.36762,0.239130,0.5,0.033333,0.379452,0.373621,0.361020,0.348419,-0.000942,-0.002154,53.338130,74.415452,2024-11-17 17:00:00,25,DOGEUSDT


In [5]:
front=['y','ds','symbol']
exo_gen_cols=data.filter(like='feature_').columns.tolist()
cols=front+[c for c in exo_gen_cols if c not in front]

train_data=train_data[cols]
test_data=test_data[cols]
# train_data
# data[id_col].unique()
# 

In [6]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [7]:
forecast_horizon=6
backend='optuna'
BiTCN_config=AutoBiTCN.get_default_config(h=forecast_horizon,backend=backend)

TFT_config=AutoTFT.get_default_config(h=forecast_horizon,backend=backend)
NBEATS_config=AutoNBEATS.get_default_config(h=forecast_horizon,backend=backend)

In [8]:
# BiTCN_MODEL= AutoBiTCN(h=forecast_horizon,
#                   loss=RMSE(),
#                   config=BiTCN_config,
#                   search_alg=optuna.samplers.TPESampler(),
#                   backend='optuna',
#                   num_samples=10)

TFT_MODEL= AutoTFT(h=forecast_horizon,
                  loss=RMSE(),
                  config=TFT_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

NBEATS_MODEL= AutoNBEATS(h=forecast_horizon,
                  loss=RMSE(),
                  config=NBEATS_config,
                  search_alg=optuna.samplers.TPESampler(),
                  backend='optuna',
                  num_samples=10)

models=[TFT_MODEL,NBEATS_MODEL]

In [9]:

model = NeuralForecast(models=models,
                    local_scaler_type='robust',
                    
                     freq='1h')

In [10]:
model.fit(train_data,
        val_size=horizon,
        time_col='ds',                    
        target_col='y',
        id_col=id_col,
        
        )


Seed set to 5
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | RMSE                     | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 256    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 154 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 63.7 K | train
6 | output_adapter          | Linear                   | 65     | train
-----------------------------------------------------------------------------
219 K     Trainable params
0         Non-trainable params
219 K     Total params
0.876     Total estimated model params size (MB)
88      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


In [11]:
model.save(save_path,overwrite=True)
save_path

'DOGEUSDTForecastingModel/'

In [12]:
test_dates=test_data.sort_values('ds')['ds'].unique()
last_date=train_data['ds'].unique()[-1]

In [13]:
test_data

,y,ds,symbol,feature_hour_of_day,feature_day_of_week,feature_day_of_month,feature_day_of_year,feature_BBANDS_BB_UPPER,feature_BBANDS_BB_MIDDLE,feature_BBANDS_BB_LOWER,feature_MACD_MACD,feature_MACD_SIGNAL,feature_14_period_RSI,feature_14_period_STOCH_%K
0,0.14252,2024-10-22 10:00:00,DOGEUSDT,-0.065217,-0.333333,0.200000,0.308219,0.146694,0.143549,0.140404,0.000151,0.000318,45.132439,26.675787
1,0.14325,2024-10-22 11:00:00,DOGEUSDT,-0.021739,-0.333333,0.200000,0.308219,0.146638,0.143635,0.140633,0.000083,0.000271,48.042450,36.662107
2,0.14196,2024-10-22 12:00:00,DOGEUSDT,0.021739,-0.333333,0.200000,0.308219,0.146459,0.143706,0.140953,-0.000074,0.000202,43.637967,19.015048
3,0.14086,2024-10-22 13:00:00,DOGEUSDT,0.065217,-0.333333,0.200000,0.308219,0.146360,0.143731,0.141103,-0.000284,0.000105,40.249394,7.019868
4,0.13906,2024-10-22 14:00:00,DOGEUSDT,0.108696,-0.333333,0.200000,0.308219,0.146882,0.143527,0.140171,-0.000589,-0.000034,35.404588,7.157895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,0.36143,2024-11-17 15:00:00,DOGEUSDT,0.152174,0.500000,0.033333,0.379452,0.374743,0.361372,0.348002,-0.001727,-0.002703,47.544678,60.921502
630,0.36372,2024-11-17 16:00:00,DOGEUSDT,0.195652,0.500000,0.033333,0.379452,0.373451,0.360964,0.348477,-0.001471,-0.002457,49.746258,67.434585
631,0.36762,2024-11-17 17:00:00,DOGEUSDT,0.239130,0.500000,0.033333,0.379452,0.373621,0.361020,0.348419,-0.000942,-0.002154,53.338130,74.415452
632,0.36722,2024-11-17 18:00:00,DOGEUSDT,0.282609,0.500000,0.033333,0.379452,0.373282,0.360922,0.348561,-0.000549,-0.001833,52.920342,71.454219
